In [0]:
# file location and type
file_location = "/FileStore/tables/tips.csv"
file_type = "csv"

# The applies options are for CSV files. For other file types, these will be ignored
df = spark.read.csv(file_location, header = True, inferSchema=True)

In [0]:
# Take a look at the df
df.show()

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|
|     18.43| 3.0|  Male|    No|Sun|Dinner|   4|
|     14.83|3.02|Female|    No|Sun|Dinner|   2|
|     21.58|3.92|  Male|    No|Sun|Dinner|   2|
|     10.33|1.67|Female|    No|Sun|Dinner|   3|
|     16.29|3.71|  Male|    No|Sun|Dinne

In [0]:
# Taking a look at data types
df.printSchema()

root
 |-- total_bill: double (nullable = true)
 |-- tip: double (nullable = true)
 |-- sex: string (nullable = true)
 |-- smoker: string (nullable = true)
 |-- day: string (nullable = true)
 |-- time: string (nullable = true)
 |-- size: integer (nullable = true)



In [0]:
df.columns

Out[6]: ['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size']

In [0]:
# Handling categorical features
from pyspark.ml.feature import StringIndexer

In [0]:
# Try to encode 1 column
indexer = StringIndexer(inputCol="sex", outputCol="sex_encoded")
df_r = indexer.fit(df).transform(df)
df_r.show()

+----------+----+------+------+---+------+----+-----------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_encoded|
+----------+----+------+------+---+------+----+-----------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|        1.0|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|        0.0|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|        0.0|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|        0.0|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|        1.0|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|        0.0|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|        0.0|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|        0.0|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|        0.0|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|        0.0|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|        0.0|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|        1.0|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|        0.0|
|     18.43| 3.0|  Male|    No|Sun|Dinne

In [0]:
# Try to encode multiple columns
indexer = StringIndexer(inputCols = ["smoker","day" ,"time"],
                        outputCols =["smoker_encoded", "day_encoded", "time_encoded"])

df_enc = indexer.fit(df_r).transform(df_r)
df_enc.show()


+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_encoded|smoker_encoded|day_encoded|time_encoded|
+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|        1.0|           0.0|        1.0|         0.0|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|        0.0|           0.0|        1.0|         0.0|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|        1.0|           0.0|        1.0|         0.0|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|        0.0|           0.0|        1.0|         0.0|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|        0.0|           0.0|        1.0|         0.0|


In [0]:
df_enc.columns

Out[13]: ['total_bill',
 'tip',
 'sex',
 'smoker',
 'day',
 'time',
 'size',
 'sex_encoded',
 'smoker_encoded',
 'day_encoded',
 'time_encoded']

In [0]:
# Vector assembler : In pyspark, we need to group all the independent features together, and the dependent feature seperately

from pyspark.ml.feature import VectorAssembler

featureAssembler = VectorAssembler(inputCols=["tip", "size", "sex_encoded","smoker_encoded", "day_encoded", "time_encoded"],
                outputCol ="Independent features")

output = featureAssembler.transform(df_enc)

In [0]:
output.select("Independent features").show()

+--------------------+
|Independent features|
+--------------------+
|[1.01,2.0,1.0,0.0...|
|[1.66,3.0,0.0,0.0...|
|[3.5,3.0,0.0,0.0,...|
|[3.31,2.0,0.0,0.0...|
|[3.61,4.0,1.0,0.0...|
|[4.71,4.0,0.0,0.0...|
|[2.0,2.0,0.0,0.0,...|
|[3.12,4.0,0.0,0.0...|
|[1.96,2.0,0.0,0.0...|
|[3.23,2.0,0.0,0.0...|
|[1.71,2.0,0.0,0.0...|
|[5.0,4.0,1.0,0.0,...|
|[1.57,2.0,0.0,0.0...|
|[3.0,4.0,0.0,0.0,...|
|[3.02,2.0,1.0,0.0...|
|[3.92,2.0,0.0,0.0...|
|[1.67,3.0,1.0,0.0...|
|[3.71,3.0,0.0,0.0...|
|[3.5,3.0,1.0,0.0,...|
|(6,[0,1],[3.35,3.0])|
+--------------------+
only showing top 20 rows



In [0]:
output.show()

+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+--------------------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_encoded|smoker_encoded|day_encoded|time_encoded|Independent features|
+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+--------------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|        1.0|           0.0|        1.0|         0.0|[1.01,2.0,1.0,0.0...|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0|[1.66,3.0,0.0,0.0...|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0|[3.5,3.0,0.0,0.0,...|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|        0.0|           0.0|        1.0|         0.0|[3.31,2.0,0.0,0.0...|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|        1.0|           0.0|        1.0|         0.0|[3.61,4.0,1.0,0.0...|
|     25.29|4.71|  Male|    No|S

In [0]:
# We want to predict Total bill (The dependent feature)
finalized_data = output.select(["Independent features", "total_bill"])
finalized_data.show()

+--------------------+----------+
|Independent features|total_bill|
+--------------------+----------+
|[1.01,2.0,1.0,0.0...|     16.99|
|[1.66,3.0,0.0,0.0...|     10.34|
|[3.5,3.0,0.0,0.0,...|     21.01|
|[3.31,2.0,0.0,0.0...|     23.68|
|[3.61,4.0,1.0,0.0...|     24.59|
|[4.71,4.0,0.0,0.0...|     25.29|
|[2.0,2.0,0.0,0.0,...|      8.77|
|[3.12,4.0,0.0,0.0...|     26.88|
|[1.96,2.0,0.0,0.0...|     15.04|
|[3.23,2.0,0.0,0.0...|     14.78|
|[1.71,2.0,0.0,0.0...|     10.27|
|[5.0,4.0,1.0,0.0,...|     35.26|
|[1.57,2.0,0.0,0.0...|     15.42|
|[3.0,4.0,0.0,0.0,...|     18.43|
|[3.02,2.0,1.0,0.0...|     14.83|
|[3.92,2.0,0.0,0.0...|     21.58|
|[1.67,3.0,1.0,0.0...|     10.33|
|[3.71,3.0,0.0,0.0...|     16.29|
|[3.5,3.0,1.0,0.0,...|     16.97|
|(6,[0,1],[3.35,3.0])|     20.65|
+--------------------+----------+
only showing top 20 rows



In [0]:
from pyspark.ml.regression import LinearRegression

# train (75%), test (25%) split
train_data, test_data = finalized_data.randomSplit([.75, .25])
regressor = LinearRegression(featuresCol="Independent features", labelCol="total_bill")
regressor = regressor.fit(train_data)

In [0]:
regressor.coefficients

Out[23]: DenseVector([3.3096, 4.0431, -0.494, 2.6708, -0.0615, -0.4724])

In [0]:
regressor.intercept

Out[24]: -0.9352738354123662

In [0]:
# Evaluate the model on test
pred_results = regressor.evaluate(test_data)

In [0]:
# Final comparaison
pred_results.predictions.show()

+--------------------+----------+------------------+
|Independent features|total_bill|        prediction|
+--------------------+----------+------------------+
|(6,[0,1],[1.75,2.0])|     17.82|12.942856457571317|
| (6,[0,1],[2.0,2.0])|     12.69|13.770264258244335|
|(6,[0,1],[2.01,2.0])|     20.23|13.803360570271256|
|(6,[0,1],[2.34,4.0])|     17.81|22.981814555432194|
| (6,[0,1],[3.0,4.0])|     20.45|25.166171149208964|
|(6,[0,1],[3.27,2.0])|     17.78|17.973495885663265|
|[1.17,2.0,0.0,1.0...|     32.83|13.694067839237738|
|[1.25,2.0,1.0,0.0...|      8.51|10.198651831245293|
|[1.32,2.0,0.0,0.0...|      9.68|11.458236190796221|
|[1.48,2.0,0.0,0.0...|      8.52|11.453863845996393|
|[1.5,2.0,0.0,0.0,...|     19.08|11.520056470050235|
|[1.5,2.0,0.0,1.0,...|     12.03|14.601809587273605|
|[1.57,2.0,0.0,0.0...|     15.42|12.285643991469238|
|[1.64,2.0,0.0,1.0...|     15.36| 15.24959450450301|
|[1.71,2.0,0.0,0.0...|     10.27|12.748992359846127|
|[1.96,2.0,0.0,0.0...|     15.04|13.5764001605

In [0]:
#----Performance Metrics------

In [0]:
# R^2
pred_results.r2

Out[27]: 0.30399277646858325

In [0]:
# Mean absolute Error (MAE)
pred_results.meanAbsoluteError

Out[28]: 4.8647126188570295

In [0]:
# Mean Squared error(MSE)
pred_results.meanSquaredError

Out[29]: 40.364683645001556

In [0]:
regressor.save

Out[32]: <bound method MLWritable.save of LinearRegressionModel: uid=LinearRegression_b299081d079e, numFeatures=6>